In [118]:
import matplotlib
matplotlib.use('Tkagg')
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdRGroupDecomposition
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdDepictor

import pandas as pd
import numpy as np
import math
from pathlib import Path
from typing import List, Tuple, Union,Optional
import itertools



In [119]:
# read the amino acids
f = r'amino_acids.smiles'
amino_acids = pd.read_csv(f, sep='|', header=0)
amino_acids['mol'] = amino_acids['smiles'].apply(Chem.MolFromSmiles)
amino_acids = amino_acids.apply(lambda row: (row['mol'].SetProp('_Name', row['name']), row)[1], axis='columns')

In [120]:
print(Chem.MolToMolBlock(amino_acids['mol'].iloc[0]))

L-Alanine
     RDKit          2D

  6  5  0  0  0  0  0  0  0  0999 V2000
   -0.4320    1.2504    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4328    0.2504    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.2992   -0.2490    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.4328   -0.2504    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4320   -1.2504    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    1.2992    0.2490    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
  2  1  1  1
  2  3  1  0
  2  4  1  0
  4  5  1  0
  4  6  2  0
M  END



In [129]:
def create_mol_grid(mols: List[Chem.Mol],
                    legends: Optional[List[str]],
                    impath: Path,
                    legend_offset: float = 0.1,
                    n_cols: int = 4,
                    figsize: Tuple[int] = (4, 4),
                    dpi: int = 300) -> None:
    '''
    It creates a gid of molecular structure images and saves the resulting
    figure.

    :param mols: an array of RDKit molecules
    :param legends: the legends to display under the molecules
    :param impath: the path to save the generated figure
    :param legend_offset: vertical offset of the legend
    :param n_cols: number of columns in the molecular image grid
    :param figsize: figure size
    :param dpi: figure resolution
    :return:
    '''
    # set interactivity to off
    plt.ioff()
    n_rows = math.ceil(len(mols) / n_cols)
    fig = plt.figure(figsize=figsize, dpi=dpi)
    fig.subplots_adjust()
    axs = fig.subplots(nrows=n_rows, ncols=n_cols,
                       gridspec_kw={'wspace': 0.01, 'hspace': 0.01})
    for i_mol, ax in enumerate(np.atleast_1d(axs).flatten()):
        if i_mol < len(mols):
            plt.setp(ax, frame_on=False)
            plt.setp(plt.getp(ax, 'xaxis'), visible=False)
            plt.setp(plt.getp(ax, 'yaxis'), visible=False)
            # set the title
            if legends:
                mol_title = legends[i_mol]
                h = ax.text(0.5, legend_offset, mol_title, ha="center", va="center",
                            rotation=0, size=2.0, bbox=None,
                            transform=ax.transAxes)
            im = Chem.Draw.MolToImage(mols[i_mol])
            ax.imshow(im)
        else:
            plt.setp(ax, visible=False)
    fig.savefig(impath, bbox_inches='tight')
    plt.close(fig)
    # set interactivity to on
    plt.ion()


# visualise the amino acids
mols = amino_acids['mol'].to_list()
legends = (amino_acids['name']+'\ncommon: '+amino_acids['common']).to_list()
legend_offset = -0.1
impath = Path('images') / 'amino_acids.png'
n_cols = 5
figsize = (4, 5)
dpi = 600
create_mol_grid(mols, legends, impath, legend_offset, n_cols, figsize, dpi)
plt.show()

In [142]:
# R-group decomposition settings
ps = rdRGroupDecomposition.RGroupDecompositionParameters()
ps.onlyMatchAtRGroups = True

# set the core with explicit R group lables
amino_acid_backbone = '[*:1][C@H](N[*:2])C(O)=O'
amino_acid_backbone = Chem.MolFromSmiles(amino_acid_backbone)

# carry out the R-group decomposition
rgd, fails = rdRGroupDecomposition.RGroupDecompose([amino_acid_backbone], amino_acids['mol'].to_list(), options=ps)
successes = [i_mol for i_mol in range(len(amino_acids)) if i_mol not in fails]

# post-process the results for visualisation
RDecomposition_results = pd.concat([amino_acids.iloc[successes]['mol'].reset_index(drop=True), pd.DataFrame(rgd).iloc[:, 1:]], axis='columns', sort=False)
mols = RDecomposition_results.stack().to_list()
legends = amino_acids.iloc[successes]['name'].to_frame().assign(R1='R1', R2='R2').stack().to_list()
legend_offset = 0.
impath = Path('images') / 'amino_acids_R_decomposed.png'
n_cols = 6
figsize = (4, 8)
dpi = 600
create_mol_grid(mols, legends, impath, legend_offset, n_cols, figsize, dpi)
plt.show()


[16:42:43] No core matches


In [143]:
# peptide
threonine_arginine_methionine = '[H]N[C@@H]([C@@H](C)O)C(=O)N[C@@H](CCCNC(N)=N)C(=O)N[C@@H](CCSC)C(O)=O'
threonine_arginine_methionine = Chem.MolFromSmiles(threonine_arginine_methionine)

# generate coordinates and orient canonically
rdDepictor.SetPreferCoordGen(True)
rdDepictor.Compute2DCoords(threonine_arginine_methionine)

# define peptide bond
peptide_bond_moiety = Chem.MolFromSmarts('NCC(=O)NCC(=O)')
d = rdMolDraw2D.MolDraw2DCairo(width=900, height=300) # or MolDraw2DSVG to get SVGs
d.drawOptions().setHighlightColour((0.8,0.8,0.8))
# matching atoms
hit_ats = threonine_arginine_methionine.GetSubstructMatches(peptide_bond_moiety)
hit_ats = list(itertools.chain(*hit_ats))

# matching bonds, annotate the peptide bond
hit_bonds = []
for bond in threonine_arginine_methionine.GetBonds():
    aid1 = bond.GetBeginAtomIdx()
    aid2 = bond.GetEndAtomIdx()
    if aid1 in hit_ats and aid2 in hit_ats:
        hit_bonds.append(bond.GetIdx())

# prepare molecule, draw molecule and create png
rdMolDraw2D.PrepareAndDrawMolecule(d, threonine_arginine_methionine, highlightAtoms=hit_ats,
                                   highlightBonds=hit_bonds)
with open('images/threonine_arginine_methionine.png', mode='wb') as f:
    f.write(d.GetDrawingText())

In [132]:
# define the reaction
peptide_hydrolysis_reaction = rdChemReactions.ReactionFromSmarts('[N:1][C:2][C:3](=[O:4])[N:5][C:6][C:7](=[O:8])>>[N:1][C:2][C:3](=[O:4])O.[N:5][C:6][C:7](=[O:8])')

# define the reactants
reacts = (threonine_arginine_methionine,)

# apply the reaction
products = peptide_hydrolysis_reaction.RunReactants(reacts, maxProducts=1000)
products = list(itertools.chain(*products))
rdDepictor.SetPreferCoordGen(True)
for product in products:
    Chem.SanitizeMol(product)
    rdDepictor.Compute2DCoords(product)

# draw the reactants in a grid
legends = None
legend_offset = 0.1
impath = Path('images') / 'reactants.png'
n_cols = 2
figsize = (1, 1)
dpi = 600
create_mol_grid(products, legends, impath, legend_offset, n_cols, figsize, dpi)

In [144]:
# apply the hydrolysis reaction recursively
def check_identity(mol1: Chem.rdchem.Mol, mol2: Chem.rdchem.Mol) -> bool:
    '''
    Checks if two rdkit molecules are identical
    :param mol1: first molecule
    :param mol2: second molecule
    :return: True if identical and False otherwise
    '''
    return all([mol1.HasSubstructMatch(mol2, useChirality=True),
                mol2.HasSubstructMatch(mol1, useChirality=True)])


def check_if_in_list(mol: Chem.rdchem.Mol, mols: List[Chem.rdchem.Mol]) -> bool:
    '''
    Checks if a molecule is identical to molecule in a list
    :param mol: molecule to be checked if present in a list
    :param mols: list of molecules
    :return: True if molecule is identical to a molecule in a list and False otherwise
    '''
    for mol_ in mols:
        if check_identity(mol, mol_):
            return True
    else:
        return False

def is_peptide(mol: Chem.rdchem.Mol,
               known_amino_acids: List[Chem.rdchem.Mol],
               rxn: Chem.rdChemReactions.ChemicalReaction) -> Union[Chem.rdchem.Mol, bool]:
    '''
    Takes a rdkit molecule and the definition of a peptide bond hydrolysis reaction and examines if the molecule is
    a linear peptide by applying the reaction recursively
    :param mol: molecule that will be checked if it is a peptide or not
    :param known_amino_acids: list of known amino acid molecules
    :param rxn: peptide hydrolysis reaction
    :return: True if molecule is a peptide is and False otherwise
    '''
    print(f'processing structure {Chem.MolToSmiles(mol)}')

    # check if peptide is already a single amino acid
    if check_if_in_list(mol, amino_acids['mol']):
        print(f'structure {Chem.MolToSmiles(mol)} is an amino acid')
        return True

    # apply the hydrolysis reaction, we only follow the first hydrolysis reaction as we
    # will apply the hydrolysis recursively anyway
    reacts = (mol,)
    products = rxn.RunReactants(reacts, maxProducts=1)
    # peptide hydrolysis could not be applied
    if not products:
        print('peptide bond hydrolysis could not be applied')
        return False
    # peptide hydrolysis was applied successfully
    else:
        # reactants of the hydrolysis
        mol1 = products[0][0]
        Chem.SanitizeMol(mol1)
        mol2 = products[0][1]
        Chem.SanitizeMol(mol2)
        print(f'applied hydrolysis reaction: {Chem.MolToSmiles(mol)} -> {Chem.MolToSmiles(mol1)} + {Chem.MolToSmiles(mol2)}')

        return all([is_peptide(mol1), is_peptide(mol2)])

In [138]:
# 5 amino acids
structures = amino_acids[['name', 'mol']].assign(type='amino acid').iloc[:5]

# di-peptide: arginine, alanine
smi = '[H]N[C@@H](CCCNC(N)=N)C(=O)N[C@@H](C)C(O)=O'
mol = Chem.MolFromSmiles(smi)
structures = pd.concat([structures, pd.Series({'name': 'arginine, alanine', 'mol': mol, 'type': 'di-peptide'}).to_frame().T], axis='index', ignore_index=True, sort=False)
# oligo-peptide: arginine, alanine, threonine, methionine
smi = '[H]N[C@@H](CCCNC(N)=N)C(=O)N[C@@H](C)C(=O)N[C@@H]([C@@H](C)O)C(=O)N[C@@H](CCSC)C(O)=O'
mol = Chem.MolFromSmiles(smi)
structures = pd.concat([structures, pd.Series({'name': 'arginine, alanine, threonine, methionine', 'mol': mol, 'type': 'oligo-peptide'}).to_frame().T], axis='index', ignore_index=True, sort=False)

# longer-peptide: ATTAMSSTA
smi = 'CSCC[C@H](NC(=O)[C@H](C)NC(=O)[C@@H](NC(=O)[C@@H](NC(=O)[C@H](C)N)[C@@H](C)O)' \
      '[C@@H](C)O)C(=O)N[C@@H](CO)C(=O)N[C@@H](CO)C(=O)N[C@@H]([C@@H](C)O)C(=O)N[C@@H](C)C(O)=O'
mol = Chem.MolFromSmiles(smi)
structures = pd.concat([structures, pd.Series({'name': 'ATTAMSSTA', 'mol': mol, 'type': 'longer peptide'}).to_frame().T], axis='index', ignore_index=True, sort=False)

# non-peptide 1
smi = '[H]N[C@@H](CCCNC(N)=N)C(=O)N[C@@H](C)C(O)'
mol = Chem.MolFromSmiles(smi)
structures = pd.concat([structures, pd.Series({'name': 'non-peptide 1', 'mol': mol, 'type': 'non-peptide'}).to_frame().T], axis='index', ignore_index=True, sort=False)

# non-peptide 2
smi = '[H]N[C@@H](CCCNC(N)=N)CC(=O)N[C@@H](C)C(O)=O'
mol = Chem.MolFromSmiles(smi)
structures = pd.concat([structures, pd.Series({'name': 'non-peptide 2', 'mol': mol, 'type': 'non-peptide'}).to_frame().T], axis='index', ignore_index=True, sort=False)

structures

,name,mol,type
0,L-Alanine,<rdkit.Chem.rdchem.Mol object at 0x0000028B01C...,amino acid
1,L-Arginine,<rdkit.Chem.rdchem.Mol object at 0x0000028B01C...,amino acid
2,L-Asparagine,<rdkit.Chem.rdchem.Mol object at 0x0000028B0AA...,amino acid
3,L-Aspartic acid,<rdkit.Chem.rdchem.Mol object at 0x0000028B0AA...,amino acid
4,L-Cysteine,<rdkit.Chem.rdchem.Mol object at 0x0000028B0AA...,amino acid
5,"arginine, alanine",<rdkit.Chem.rdchem.Mol object at 0x0000028B195...,di-peptide
6,"arginine, alanine, threonine, methionine",<rdkit.Chem.rdchem.Mol object at 0x0000028B195...,oligo-peptide
7,ATTAMSSTA,<rdkit.Chem.rdchem.Mol object at 0x0000028B195...,longer peptide
8,non-peptide 1,<rdkit.Chem.rdchem.Mol object at 0x0000028B195...,non-peptide
9,non-peptide 2,<rdkit.Chem.rdchem.Mol object at 0x0000028B195...,non-peptide


In [140]:
print(structures)

                                       name  \
0                                 L-Alanine   
1                                L-Arginine   
2                              L-Asparagine   
3                           L-Aspartic acid   
4                                L-Cysteine   
5                         arginine, alanine   
6  arginine, alanine, threonine, methionine   
7                                 ATTAMSSTA   
8                             non-peptide 1   
9                             non-peptide 2   

                                                 mol            type  
0  <rdkit.Chem.rdchem.Mol object at 0x0000028B01C...      amino acid  
1  <rdkit.Chem.rdchem.Mol object at 0x0000028B01C...      amino acid  
2  <rdkit.Chem.rdchem.Mol object at 0x0000028B0AA...      amino acid  
3  <rdkit.Chem.rdchem.Mol object at 0x0000028B0AA...      amino acid  
4  <rdkit.Chem.rdchem.Mol object at 0x0000028B0AA...      amino acid  
5  <rdkit.Chem.rdchem.Mol object at 0x0000028B195...    

In [141]:
import pandas as pd

# Create sample data (assuming 'mol' is a list of molecules and 'type' is a list of molecule types)
structures = pd.DataFrame({'mol': [['C', 'O', 'C', 'O'], ['C', 'C', 'C', 'N']],
                           'type': ['peptide', 'non-peptide']})

# Assuming is_peptide() is a function that takes a molecule and returns True if it's a peptide, False otherwise
def is_peptide(mol):
    # Replace this with your actual peptide identification logic
    return len(mol) >= 3 and mol[0] == 'C' and mol[-1] == 'O'

# Apply the is_peptide() function to the 'mol' column and store the results in the 'result' column
structures['result'] = structures['mol'].apply(is_peptide)

# Create legends for visualization (assuming create_mol_grid() is a visualization function)
legends = (structures['type'] + '\nis peptide: ' + structures['result'].astype(str)).to_list()

# Visualize the results (replace with your actual visualization code)
# ... (assuming create_mol_grid() exists and takes necessary arguments)

print(structures)  # Print the modified DataFrame

            mol         type  result
0  [C, O, C, O]      peptide    True
1  [C, C, C, N]  non-peptide   False
